# `PYQG` : Generating Subgrid Forcing Datasets

## 1 - Introduction & Setup
In this notebook, we'll document and showcase datasets for learning subgrid forcing parameterizations, generated with [`pyqg`](https://pyqg.readthedocs.io/en/latest/).